In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from random import randrange
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import log_loss
from datetime import datetime
import autosklearn.classification
from sklearn.metrics import ConfusionMatrixDisplay
from joblib import dump
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import sys
sys.path.append('/home/krzysiek/drivendata/drivendata_competitions/functions')
from eda_automations import *
from sklearn.model_selection import train_test_split
import random as r
pd.set_option('display.max_columns', None)

In [54]:
df = pd.read_csv('https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv')

In [55]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [56]:
labels_data = pd.read_csv('https://github.com/KCristopher/drivendata_competitions/raw/main/data/training_set_labels.csv')

In [57]:
labels_data.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


The data contains the info about waterpoints in Tanzania. Each observations correspond to some waterpoint and is described in terms of the variables such as the height at which the waterpoint is located, type of source of the water (for example rain or spring) or the institution which built the well.  

We hope that we would be able to discern which waterponits are functional, which are not, and which needs to be repaired by examining the levels of all these different variables.

In [58]:
labels_data.index = labels_data['id']
del labels_data['id']

In [59]:
labels_data.head(3)

,status_group
id,
69572,functional
8776,functional
34310,functional


In [60]:
y = labels_data.copy()
X = df.copy()

Splitting the data into training and testing sets :

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Exploring training data

<center>Checking the general structure of the data, data types and  missing values :</center>

In [62]:
summarize_df(X_train)


Glance at the data : 



,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
3607,454,50.0,2013-02-27,Dmdd,2092,DMDD,35.426020,-4.227446,Narmo,0,Internal,Bashnet Kati,Manyara,21,1,Babati,Bashinet,160,True,GeoData Consultants Ltd,Water Board,NaN,True,1998,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
50870,510,0.0,2011-03-17,Cmsr,0,Gove,35.510074,-5.724555,Lukali,0,Internal,Lukali,Dodoma,1,6,Bahi,Lamaiti,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
20413,14146,0.0,2011-07-10,Kkkt,0,KKKT,32.499866,-9.081222,Mahakama,0,Lake Rukwa,Chawalikozi,Mbeya,12,6,Mbozi,Ndalambo,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
52806,47410,0.0,2011-04-12,NaN,0,NaN,34.060484,-8.830208,Shule Ya Msingi Chosi A,0,Rufiji,Shuleni,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe
50091,1288,300.0,2011-04-05,Ki,1023,Ki,37.032690,-6.040787,Kwa Mjowe,0,Wami / Ruvu,Ngholong,Morogoro,5,1,Kilosa,Chakwale,120,True,GeoData Consultants Ltd,VWC,NaN,True,1997,other,other,other,vwc,user-group,pay when scheme fails,on failure,salty,salty,enough,enough,shallow well,shallow well,groundwater,other,other



Data types, dimension of the data : 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 3607 to 56422
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     47520 non-null  int64  
 1   amount_tsh             47520 non-null  float64
 2   date_recorded          47520 non-null  object 
 3   funder                 44644 non-null  object 
 4   gps_height             47520 non-null  int64  
 5   installer              44631 non-null  object 
 6   longitude              47520 non-null  float64
 7   latitude               47520 non-null  float64
 8   wpt_name               47520 non-null  object 
 9   num_private            47520 non-null  int64  
 10  basin                  47520 non-null  object 
 11  subvillage             47224 non-null  object 
 12  region                 47520 non-null  object 
 13  region_code            47520 non-null  int64  
 14  district_cod

None


Missing values across variables : 



scheme_name              22523
scheme_management         3102
installer                 2889
funder                    2876
public_meeting            2689
permit                    2439
subvillage                 296
management_group             0
waterpoint_type              0
extraction_type              0
extraction_type_group        0
source_class                 0
extraction_type_class        0
management                   0
source_type                  0
source                       0
payment_type                 0
water_quality                0
construction_year            0
quality_group                0
quantity                     0
quantity_group               0
payment                      0
id                           0
amount_tsh                   0
recorded_by                  0
population                   0
ward                         0
lga                          0
district_code                0
region_code                  0
region                       0
basin   


Missing values - % of the data : 


scheme_name              47.40
scheme_management         6.53
installer                 6.08
funder                    6.05
public_meeting            5.66
permit                    5.13
subvillage                0.62
management_group          0.00
waterpoint_type           0.00
extraction_type           0.00
extraction_type_group     0.00
source_class              0.00
extraction_type_class     0.00
management                0.00
source_type               0.00
source                    0.00
payment_type              0.00
water_quality             0.00
construction_year         0.00
quality_group             0.00
quantity                  0.00
quantity_group            0.00
payment                   0.00
id                        0.00
amount_tsh                0.00
recorded_by               0.00
population                0.00
ward                      0.00
lga                       0.00
district_code             0.00
region_code               0.00
region                    0.00
basin   

In [63]:
codes = [c for c in X_train.columns if 'code' in c]
codes

['region_code', 'district_code']

<center>Getting numeric summaries of the data</center>

In [64]:
summarize_numeric_data(X_train, exclude = ['id'] + codes)


The correlation between the variables in the dataset :



,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year
amount_tsh,1.00,0.07,0.02,-0.05,0.00,0.01,0.06
gps_height,0.07,1.00,0.15,-0.04,0.01,0.13,0.66
longitude,0.02,0.15,1.00,-0.43,0.02,0.09,0.40
latitude,-0.05,-0.04,-0.43,1.00,0.01,-0.02,-0.25
num_private,0.00,0.01,0.02,0.01,1.00,0.00,0.03
population,0.01,0.13,0.09,-0.02,0.00,1.00,0.26
construction_year,0.06,0.66,0.40,-0.25,0.03,0.26,1.00



Descriptive statistics of numeric data : 



,amount_tsh,gps_height,longitude,latitude,num_private,population,construction_year
count,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00
mean,322.05,668.75,34.09,-5.71,0.50,179.53,1303.35
std,3200.62,692.97,6.54,2.94,13.25,472.77,950.76
min,0.00,-63.00,0.00,-11.65,0.00,0.00,0.00
25%,0.00,0.00,33.08,-8.53,0.00,0.00,0.00
50%,0.00,370.00,34.91,-5.02,0.00,25.00,1986.00
75%,20.00,1320.00,37.18,-3.33,0.00,213.00,2004.00
max,350000.00,2770.00,40.35,-0.00,1776.00,30500.00,2013.00


In [65]:
look_at_variables_values(X_train)


Unique values from id, limited to 30

35563    17378
23497    40971
23268    15247
14844    36192
42261    55367
36343    59772
1530     46403
36879     9367
42623    30402
43791    45749
1548     48300
42067    21549
3687     65956
44990    62949
11840    66549
7566     10249
27423    38649
25994    36645
37765    23406
23144     3829
27603    28563
40658    51769
13774    10370
27352    70555
23342    56823
45641    19841
23930    66685
47165    43837
36912     4993
34338    51600
dtype: int64


Unique values from amount_tsh, limited to 30

66    350000.00
52      7000.00
7         30.00
72         0.20
16        15.00
18         5.00
57     70000.00
32       750.00
69     45000.00
13      1500.00
2        300.00
93         0.25
46        70.00
58     50000.00
28       450.00
75     60000.00
27      8500.00
95      1400.00
5        250.00
0         50.00
64      6300.00
54      7500.00
83     13000.00
68         9.00
94        59.00
23      2500.00
48      3500.00
44     20000.00
39

8                   other
5                     dam
4                hand dtw
2                   river
6    rainwater harvesting
9                 unknown
7                    lake
1            shallow well
0                  spring
3             machine dbh
dtype: object


Unique values from source_type, limited to 30

0                  spring
1            shallow well
6                   other
2              river/lake
5    rainwater harvesting
4                     dam
3                borehole
dtype: object


Unique values from source_class, limited to 30

0    groundwater
1        surface
2        unknown
dtype: object


Unique values from waterpoint_type, limited to 30

2                          other
0             communal standpipe
5                  cattle trough
4                improved spring
3    communal standpipe multiple
6                            dam
1                      hand pump
dtype: object


Unique values from waterpoint_type_group, limited to 30

5        

We can see that there is one entry of 0 in the column construction_year.

In [66]:
X_train[X_train.construction_year == 0].shape

(16503, 40)

In [67]:
X_train.columns.tolist()

['id',
 'amount_tsh',
 'date_recorded',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'num_private',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [68]:
cols_subset = ['ward',
 'population',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality']

In [69]:
X_train[X_train.construction_year == 0][cols_subset].sample(n = 100).head(50)

,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality
7871,Utengule Usongwe,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,gravity,gravity,gravity,wug,user-group,pay when scheme fails,on failure,soft
36665,Mwantini,0,False,GeoData Consultants Ltd,Parastatal,NaN,True,0,other,other,other,parastatal,parastatal,never pay,never pay,soft
12363,Usagara,0,False,GeoData Consultants Ltd,VWC,NaN,True,0,other,other,other,vwc,user-group,never pay,never pay,soft
42705,Lufilyo,0,True,GeoData Consultants Ltd,Parastatal,K,NaN,0,gravity,gravity,gravity,parastatal,parastatal,unknown,unknown,soft
20760,Mwaniko,0,NaN,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft
16184,Mwananyili,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,mono,mono,motorpump,vwc,user-group,never pay,never pay,soft
41059,Kilago,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky
39461,Upuge,0,True,GeoData Consultants Ltd,VWC,Upuge water supply,True,0,mono,mono,motorpump,vwc,user-group,never pay,never pay,soft
25157,Bugarama,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky
6627,Kalebezo,0,True,GeoData Consultants Ltd,Water Board,LumeyaKalebezoNyehunge water project,True,0,mono,mono,motorpump,water board,user-group,never pay,never pay,soft


In [70]:
def count_values(df, value):
    """
    Returns a pandas DataFrame with the names of the original features in the index and the number of
    occurrences of the value as the values of the first column and the same as the percentage of all
    observations in the second column.

    If the passed value is boolean, compares with boolean values in the DataFrame. If the passed value
    is a string, compares with object (string) values in the DataFrame. Otherwise, only compares with
    values of the same data type.

    Parameters:
    -----------
    df: pandas DataFrame.

    value : str / int / float / bool.
        The value to be counted.

    Returns:
    --------
    pandas DataFrame.
    """

    if isinstance(value, bool):
        value_mask = df.apply(lambda x: isinstance(x[0], bool), axis=0)
    elif isinstance(value, str):
        value_mask = df.dtypes == 'object'
    elif isinstance(value, int):
        value_mask = df.dtypes == 'int64'
    elif isinstance(value, float):
        value_mask = df.dtypes == 'float64'
    else:
        raise TypeError("Input value must be of type bool, str, int, or float.")

    count_series = df.loc[:, value_mask].eq(value).sum().sort_values(ascending=False)
    percent_series = count_series / len(df) * 100
    percent_series = percent_series.round(1)

    data = {
        f"Count of {value}": count_series,
        f"{value}'s as % of Total": percent_series
    }
    result_df = pd.DataFrame(data)
    result_df.index.name = 'Feature'
    result_df.sort_values(by=f"Count of {value}", ascending=False, inplace=True)

    return result_df[result_df['Count of {}'.format(value)] > 0]


In [71]:
X_train[['permit', 'scheme_management']]

,permit,scheme_management
3607,True,Water Board
50870,True,VWC
20413,False,VWC
52806,True,VWC
50091,True,VWC
...,...,...
54343,True,Water Board
38158,False,VWC
860,False,Private operator
15795,True,WUG


<center>Examining potentially not obvious missing values</center>

In [72]:
count_values(df = X_train, value = 0)

,Count of 0,0's as % of Total
Feature,,
num_private,46903,98.7
population,17048,35.9
construction_year,16503,34.7
gps_height,16275,34.2
district_code,19,0.0
id,1,0.0


In [73]:
X_train.num_private.value_counts(normalize = True).round(3).head(3)

0    0.987
6    0.001
1    0.001
Name: num_private, dtype: float64

In [74]:
X_train[X_train.num_private == 0].sample(n = 10)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
45674,33627,0.0,2013-01-28,Tasaf,356,TASAF,38.361339,-9.419367e+00,Mkuyuni,0,Ruvuma / Southern Coast,Nalungombe,Lindi,80,43,Liwale,Kimambi,350,NaN,GeoData Consultants Ltd,VWC,NaN,True,2007,india mark ii,india mark ii,handpump,vwc,user-group,unknown,unknown,salty,salty,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
12470,62496,20.0,2013-03-25,Magadini-makiwaru Water,1353,Magadini-Makiwaru wa,37.042011,-3.173119e+00,Kwa Edward Mushi,0,Pangani,Kilingi,Kilimanjaro,3,7,Siha,Siha Kati,1,True,GeoData Consultants Ltd,Water Board,BL Sanya Hoi,True,2001,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
16332,18680,500.0,2013-02-14,Women For Partnership,1712,District Council,37.927210,-4.411243e+00,Hemkisinga,0,Pangani,Mturo,Kilimanjaro,3,3,Same,Bwambo,156,True,GeoData Consultants Ltd,VWC,Mwembe Water Supply,True,2012,gravity,gravity,gravity,vwc,user-group,pay when scheme fails,on failure,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
5108,12992,0.0,2011-07-12,Jica,0,DWE,31.176976,-1.813252e+00,Kwaemmanueli,0,Lake Victoria,Omukimeya,Kagera,18,1,Karagwe,Rugu,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
40011,71798,0.0,2011-07-27,Hesawa,0,Hesawa,0.000000,-2.000000e-08,Bombani,0,Lake Victoria,Mwamatumba,Mwanza,19,6,Geita,Lubanga,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,other,other,other,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,shallow well,shallow well,groundwater,other,other
58402,46642,0.0,2011-07-30,Ka,0,KA,31.568437,-1.714923e+00,Kwa Clemence,0,Lake Victoria,Kayoro,Kagera,18,3,Muleba,Kishanda,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
35022,71910,0.0,2013-01-22,Mdrdp,1693,MDRDP,35.556430,-3.805739e+00,Kwa Joseph Geheri,0,Internal,Ants B,Manyara,21,3,Mbulu,Bargish,350,NaN,GeoData Consultants Ltd,WUG,NaN,True,2004,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
8720,69090,20.0,2013-02-18,Jaica,-6,Jaica,40.167112,-1.065451e+01,Tanita,0,Ruvuma / Southern Coast,Tanita,Mtwara,99,1,Mtwara Rural,Kitaya,200,True,GeoData Consultants Ltd,NaN,NaN,True,2006,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
2486,469,25.0,2011-03-07,Donor,290,Do,38.238568,-6.179919e+00,Kwasungwini,0,Wami / Ruvu,Kwedigongo,Pwani,6,1,Bagamoyo,Mbwewe,2300,True,GeoData Consultants Ltd,VWC,NaN,False,2010,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
34326,64842,500.0,2011-07-07,European Union,1786,Accra,34.822766,-9.041447e+00,Kwa Andreasi Mtivike,0,Rufiji,Lunguya,Iringa,11,4,Njombe,Mtwango,95,True,GeoData Consultants Ltd,WUG,Mtwango water supplied sche,True,2011,gravity,gravity,gravity,wug,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


Regardless whether num_private, for which description is not available, can really be 0 or it is a missing value, we can safely remove it, because the feature which has the same value in 99 % of cases wouldn't be very useful anyway. There is a big chance that t is a missing value, too.

In [75]:
del X_train['num_private']

In [76]:
X_train.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
3607,454,50.0,2013-02-27,Dmdd,2092,DMDD,35.426020,-4.227446,Narmo,Internal,Bashnet Kati,Manyara,21,1,Babati,Bashinet,160,True,GeoData Consultants Ltd,Water Board,NaN,True,1998,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
50870,510,0.0,2011-03-17,Cmsr,0,Gove,35.510074,-5.724555,Lukali,Internal,Lukali,Dodoma,1,6,Bahi,Lamaiti,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [77]:
count_values(df = X_train, value = '-')

,Count of -,-'s as % of Total
Feature,,
installer,3,0.0


In [78]:
X_train[X_train.installer == '-']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
10217,42616,0.0,2011-08-03,Kalebejo Parish,0,-,32.356645,-2.499427,Kalebejo Parish,Lake Victoria,Nyang'Ombe,Mwanza,19,5,Sengerema,Kalebezo,0,NaN,GeoData Consultants Ltd,Private operator,None,True,0,swn 80,swn 80,handpump,private operator,commercial,never pay,never pay,salty abandoned,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
25769,21336,0.0,2011-07-26,Government Of Tanzania,0,-,32.674665,-2.506721,Health Center,Lake Victoria,Katungulu Center,Mwanza,19,5,Sengerema,Katunguru,0,True,GeoData Consultants Ltd,VWC,water supply in katungulu,True,0,other,other,other,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,lake,river/lake,surface,communal standpipe,communal standpipe
20968,10873,0.0,2011-07-26,Government Of Tanzania,0,-,32.677150,-2.508912,Kwa Madebele,Lake Victoria,Katungulu Center,Mwanza,19,5,Sengerema,Katunguru,0,True,GeoData Consultants Ltd,VWC,water supply Katungulu,True,0,other,other,other,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,lake,river/lake,surface,communal standpipe,communal standpipe


hyphens ('-') must represent NaNs.

In [79]:
X_train.loc[ : , 'installer'] = X_train['installer'].apply(lambda x : np.nan if x == '-' else x)

/tmp/ipykernel_1150/403331154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[ : , 'installer'] = X_train['installer'].apply(lambda x : np.nan if x == '-' else x)


In [80]:
X_train[X_train.installer == '-']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group


In [81]:
X_train.shape

(47520, 39)

In [82]:
X_train[X_train.population == 0]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
50870,510,0.0,2011-03-17,Cmsr,0,Gove,35.510074,-5.724555,Lukali,Internal,Lukali,Dodoma,1,6,Bahi,Lamaiti,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
20413,14146,0.0,2011-07-10,Kkkt,0,KKKT,32.499866,-9.081222,Mahakama,Lake Rukwa,Chawalikozi,Mbeya,12,6,Mbozi,Ndalambo,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
52806,47410,0.0,2011-04-12,NaN,0,NaN,34.060484,-8.830208,Shule Ya Msingi Chosi A,Rufiji,Shuleni,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe
16521,13095,0.0,2011-08-08,Hesawa,0,DWE,33.509112,-2.648505,Kwa Mudaba,Lake Victoria,Lumeji,Mwanza,19,2,Magu,Sukuma,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
52225,558,0.0,2013-03-01,World Vision,0,World vision,33.731347,-3.284633,Mwamagulya,Internal,Ngomeni,Shinyanga,17,2,Maswa,Busilili,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,other,other,soft,good,seasonal,seasonal,shallow well,shallow well,groundwater,hand pump,hand pump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41090,50530,0.0,2011-03-24,World Bank,0,Inter,36.644403,-6.249858,Kwa John Makago,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Iduo,0,True,GeoData Consultants Ltd,VWC,Iduo,False,0,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
44131,5457,0.0,2011-03-14,Bsf,0,DWE,35.238399,-5.172737,Machineni,Internal,Kilimba B,Dodoma,1,1,Kondoa,Ovada,0,True,GeoData Consultants Ltd,VWC,Ovad,False,0,cemo,other motorpump,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
37194,73111,0.0,2011-02-26,Kkkt,1922,Commu,34.230959,-9.369277,none,Lake Nyasa,Ikonda Bo,Iringa,11,3,Makete,Lupalilo,0,True,GeoData Consultants Ltd,VWC,Ikonda,False,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
6265,47858,0.0,2011-03-10,Bsf,0,Gove,35.547568,-6.336433,Mnadan,Internal,Kikuyu,Dodoma,1,6,Bahi,Ibugule,0,True,GeoData Consultants Ltd,VWC,Nkho,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe


In [83]:
X_train.loc[ : , 'exact_location'] = [(long, lat) for long, lat in zip(X_train.longitude, X_train.latitude)]

/tmp/ipykernel_1150/888980537.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[ : , 'exact_location'] = [(long, lat) for long, lat in zip(X_train.longitude, X_train.latitude)]


In [84]:
X_train.sample(n = 1000).groupby('exact_location')['population'].unique()

exact_location
(0.0, -2e-08)                          [0]
(29.64824669, -4.79080435)           [500]
(29.65885566, -4.82362806)           [863]
(29.679249100000003, -4.75710463)    [300]
(29.69488, -4.74613666)              [855]
                                     ...  
(39.78010514, -10.58139952)          [200]
(40.12213949, -10.21481169)          [200]
(40.19065133, -10.27330099)          [130]
(40.27985323, -10.53735322)          [300]
(40.30198716, -10.32095944)          [500]
Name: population, Length: 976, dtype: object

In [85]:
(X_train.sample(n = 1000).groupby('exact_location')['population'].nunique() == 1).all()

True

Out of 1000 random locations, each location with the same coordinates had the same population, which mean that there are no apparent isses whith the quality of population data.

In [86]:
del X_train['exact_location']

In [87]:
X_train.sample(n = 1000).groupby('subvillage')['population'].unique().head(60)

subvillage
Alosinoni                      [1]
Amani                          [0]
Ants A                       [459]
Anuru                        [135]
Arauyo                       [360]
Arusha                         [0]
Azimio                       [250]
Bagamoyo                  [210, 0]
Barabara                     [147]
Barabara 5                   [345]
Barabara 6                   [100]
Baranyota                    [300]
Batini                        [52]
Batini B                     [100]
Betania                        [1]
Binga Mjini                  [600]
Biyonza                        [0]
Bokara                      [2000]
Bomani                      [4000]
Bombamzinga                    [0]
Bombani                        [0]
Bomu                          [10]
Bramba                       [350]
Bubalaji                       [0]
Budula                         [0]
Buganda                        [0]
Bugengere                      [0]
Buguruni                     [300]
Buhaji   

Population refers to population around the well, so we might have multiple values of population in a ward or even in subvillage. Browsing for sample villages with zero poplation in the Web we find that in fact they do have a population, which makes sense, since there is little incentive to build a well for some lone wanderers in uninhabited places..

In [88]:
X_train.loc[X_train.population == 0, 'population'] = np.nan

In [89]:
X_train.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
3607,454,50.0,2013-02-27,Dmdd,2092,DMDD,35.426020,-4.227446,Narmo,Internal,Bashnet Kati,Manyara,21,1,Babati,Bashinet,160.0,True,GeoData Consultants Ltd,Water Board,NaN,True,1998,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
50870,510,0.0,2011-03-17,Cmsr,0,Gove,35.510074,-5.724555,Lukali,Internal,Lukali,Dodoma,1,6,Bahi,Lamaiti,NaN,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [90]:
X_train.population.isnull().sum()

17048

In [91]:
X_train.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
3607,454,50.0,2013-02-27,Dmdd,2092,DMDD,35.426020,-4.227446,Narmo,Internal,Bashnet Kati,Manyara,21,1,Babati,Bashinet,160.0,True,GeoData Consultants Ltd,Water Board,NaN,True,1998,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
50870,510,0.0,2011-03-17,Cmsr,0,Gove,35.510074,-5.724555,Lukali,Internal,Lukali,Dodoma,1,6,Bahi,Lamaiti,NaN,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [92]:
X_train.isnull().sum().sort_values(ascending = False) / len(X_train)

scheme_name              22523
population               17048
scheme_management         3102
installer                 2892
funder                    2876
public_meeting            2689
permit                    2439
subvillage                 296
payment_type                 0
extraction_type_class        0
management                   0
management_group             0
payment                      0
quality_group                0
water_quality                0
extraction_type              0
quantity                     0
quantity_group               0
source                       0
source_type                  0
source_class                 0
waterpoint_type              0
extraction_type_group        0
id                           0
construction_year            0
amount_tsh                   0
recorded_by                  0
ward                         0
lga                          0
district_code                0
region_code                  0
region                       0
basin   